In [96]:
!pip3 install dydx-v3-python==1.9.1
!pip3 install web3

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [97]:
from dydx3 import Client
from web3 import Web3
from pprint import pprint
from datetime import datetime, timedelta

# Testnet
from dydx3.constants import API_HOST_GOERLI
# Mainnet
from dydx3.constants import API_HOST_MAINNET

provider = "https://eth-goerli.g.alchemy.com/v2/d6_ZXhjTNOhS9YwxsRb80autxqrsCxyI"


In [98]:
# Constants
ETH_ADDRESS = "0x1Dd092B79C0c340A0EBd9B90ec190D540091Ed24"
ETH_PK = "0x12cad9378906485295cd6e9511accdede95819969d5ec19e3dea0d8eec910e60"

STARK_PK = "0422bfeac234a3a924aaf05df5dc307ac9dedf140e18544f0b2e29ecdabbd797"
DYDX_API_KEY = "1cc04d51-c231-fdf9-620a-23e28e3ff7b3"
DYDX_API_SECRET = "bHBl8cSA_x7wn9I5I920d1ALwJUr4JwxJdlrBglS"
DYDX_API_PASSPHRASE = "7paQVTFByacomo-z5swh"

# testnet
HOST = API_HOST_GOERLI

In [99]:
# Connect to DYDX 
client = Client(
    host=HOST,
    api_key_credentials= {
        "key": DYDX_API_KEY,
        "secret": DYDX_API_SECRET,
        "passphrase": DYDX_API_PASSPHRASE,
    },
    stark_private_key=STARK_PK,
    eth_private_key=ETH_PK,
    default_ethereum_address=ETH_ADDRESS,
    web3=Web3(Web3.HTTPProvider(provider))
)

In [100]:
acct = client.private.get_account()
acct_id = acct.data["account"]["id"]
quote_balance = acct.data["account"]["quoteBalance"]

print("Connected to DYDX")
print("Acct ID:", acct_id)
print("Balance:", quote_balance)

Connected to DYDX
Acct ID: 26f35227-bd91-5c5b-82ff-5c89be92e3cc
Balance: 1885.399000


In [101]:
candles = client.public.get_candles(
  market="BTC-USD",
  resolution='1HOUR',
  limit=3
)
pprint(candles.data['candles'][0])

{'baseTokenVolume': '0.0048',
 'close': '28882',
 'high': '28882',
 'low': '28866',
 'market': 'BTC-USD',
 'open': '28870',
 'resolution': '1HOUR',
 'startedAt': '2023-05-07T04:00:00.000Z',
 'startingOpenInterest': '302.9296',
 'trades': '3',
 'updatedAt': '2023-05-07T04:41:02.065Z',
 'usdVolume': '138.584'}


In [102]:
# Get postion ID
acct_response = client.private.get_account()
position_id = acct_response.data["account"]["positionId"]

server_time = client.public.get_time()
expiration = datetime.fromisoformat(server_time.data["iso"].replace("Z", "")) + timedelta(seconds=70)

# placed_order_eth = client.private.create_order(
#   position_id=position_id, # required for creating the order signature
#   market="ETH-USD",
#   side="BUY",
#   order_type="MARKET",
#   post_only=False,
#   size='0.01',
#   price='100000',
#   limit_fee='0.015',
#   expiration_epoch_seconds=expiration.timestamp(),
#   time_in_force="FOK",
#   reduce_only=False
# )

# placed_order_btc = client.private.create_order(
#   position_id=position_id, # required for creating the order signature
#   market="BTC-USD",
#   side="BUY",
#   order_type="MARKET",
#   post_only=False,
#   size='0.001',
#   price='100000',
#   limit_fee='0.015',
#   expiration_epoch_seconds=expiration.timestamp(),
#   time_in_force="FOK",
#   reduce_only=False
# )


sell_order_eth = client.private.create_order(
  position_id=position_id, # required for creating the order signature
  market="ETH-USD",
  side="SELL",
  order_type="MARKET",
  post_only=False,
  size='0.01',
  price='100000',
  limit_fee='0.015',
  expiration_epoch_seconds=expiration.timestamp(),
  time_in_force="FOK",
  reduce_only=False
)

sell_order_btc = client.private.create_order(
  position_id=position_id, # required for creating the order signature
  market="BTC-USD",
  side="SELL",
  order_type="MARKET",
  post_only=False,
  size='0.001',
  price='100000',
  limit_fee='0.015',
  expiration_epoch_seconds=expiration.timestamp(),
  time_in_force="FOK",
  reduce_only=False
)


In [103]:
pprint(sell_order_eth.data)
print("----------------------------------------")
pprint(sell_order_btc.data)

{'order': {'accountId': '26f35227-bd91-5c5b-82ff-5c89be92e3cc',
           'cancelReason': None,
           'clientId': '9003127358310393',
           'createdAt': '2023-05-07T04:45:12.086Z',
           'expiresAt': '2023-05-07T09:46:21.598Z',
           'id': '4cab1e6687c34bf5bdfa95194809c749904d30809862d6b4a3b8ed7be1b8b61',
           'market': 'ETH-USD',
           'postOnly': False,
           'price': '100000',
           'reduceOnly': False,
           'reduceOnlySize': None,
           'remainingSize': '0.01',
           'side': 'SELL',
           'size': '0.01',
           'status': 'PENDING',
           'timeInForce': 'FOK',
           'trailingPercent': None,
           'triggerPrice': None,
           'type': 'MARKET',
           'unfillableAt': None}}
----------------------------------------
{'order': {'accountId': '26f35227-bd91-5c5b-82ff-5c89be92e3cc',
           'cancelReason': None,
           'clientId': '7406933609191918',
           'createdAt': '2023-05-07T04:45:12.